In [1]:
from __future__ import division
import os
import codecs
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from itertools import combinations
import nltk
import ngrammer
import pymorphy2
#from udpipe_model import Model
from wiki_ru_wordnet import WikiWordnet
import xlrd
import glob
import artm
import numpy as np

In [2]:
def ClearTable(table):
    for i in table:
        if i[0]==i[1]:
            table[i] == False
        else:
            table[i] == 0

def Add(dic, value):
    if value in dic:
        dic[value] +=1
    else:
        dic[value] = 1

def All(dic):
    for i in dic:
        if i[0] == i[1] and dic[i] == False:
            return False
    return True

def GetDict(cluster_list, class_list):
    s = set()
    d = {}
    for x in cluster_list:
        s.add(x)
    for kek in s:
        self_d = {}
        for i in s:
            self_d[i] = 0
        for i in range(len(cluster_list)):
            if cluster_list[i] == kek:
                Add(self_d, class_list[i])
        d[kek] = dict(self_d)
    return d
def Swap(cluster_list, class_list, max_key):
    for i in range(len(cluster_list)):
        if cluster_list[i] == max_key[0]:
            cluster_list[i] = max_key[1]
        elif cluster_list[i] == max_key[1]:
            cluster_list[i] = max_key[0]
def CPS(cluster_list, class_list):
    s = set()
    for x in cluster_list:
        s.add(x)
    table = {}
    for i in combinations(list(s), 2):
        table[i] = 0
    for i in s:
        table[(i, i)] = False
    while True:
        d = GetDict(cluster_list, class_list)
        for line in table:
            if line[0] != line[1]:
                table[line] = d[line[0]][line[1]] + d[line[1]][line[0]]
            else:
                table[line] = d[line[0]][line[1]]
        max_key = None
        for line in table:
            if table[line] > table[(line[0], line[0])] + table[(line[1], line[1])]:
                if max_key == None:
                    max_key = line
                elif table[max_key] < table[line]:
                    max_key = line
        if max_key == None:
            break
        else:
            Swap(cluster_list, class_list, max_key)

    counter = 0
    for i,x in enumerate(class_list):
        if x == cluster_list[i]:
            counter += 1
    return counter/len(class_list)


In [3]:
class Parser(object):
    _data = []
    def __init__(self, data_path):
        file_list = [file for file in os.listdir(data_path) if os.path.isfile(data_path + '/' + file)]
        file_list.sort(key = lambda x: int(x[2:-4]))
        self._data = [codecs.open(data_path + '/' + file, 'r', 'utf-8').read() for file in file_list]

In [4]:
class PNExtracter(Parser):
    _negatives = []
    _positives = []
    _model = None
    _workbook = None
    
    def _Add(word, dictionary, score):
        if word in dictionary:
            dictionary[word] += score
        else:
            dictionary[word] = score
    def _Distr(dictionary, pos_dic, neg_dic):
        for word in dictionary:
            if dictionary[word] > 0:
                if word in pos_dic:
                    pos_dic[word] +=1
                else:
                    pos_dic[word] = 1
            elif dictionary[word] < 0:
                if word in neg_dic:
                    neg_dic[word] +=1
                else:
                    neg_dic[word] = 1

    def _Score(word, wordnet, workbook):
        worksheet = workbook.sheet_by_index(0)
        score = 0
        synsets = wordnet.get_synsets(word)
        if bool(synsets):
            synset = synsets[0]
            synlist = [i.lemma() for i in synset.get_words()]
        else:
            synlist = [word]  
        counter = 0
        for token in synlist:
            flag = False
            for i in range(worksheet.nrows):
                if token == worksheet.cell(i, 0).value:
                    counter +=1
                    flag = True
                    score += int(worksheet.cell(i, 1).value)
                elif flag:
                    break
        if counter:
            score /= counter
        else:
            score = 0
        return score

    def __init__(self, data_path, model_path, word_rate_path):
        Parser.__init__(self, data_path)
        self._model = Model(model_path)
        self._workbook = xlrd.open_workbook(word_rate_path)
    def get_negatives(self):
        return self._negatives
    def get_positives(self):
        return self._positives
    def extract_pn(self):
        for i, doc in enumerate(self._data):
            sentences = self._model.tokenize(doc)
            for s in sentences:
                self._model.tag(s)
                self._model.parse(s)
            conllu = self._model.write(sentences, "conllu")

            flag = False
            deps = []
            dep = []
            for line in conllu.split('\n'):
                if len(line) > 1 and line[0] == '1':
                    flag = True
                elif len(line) == 0:
                    flag = False
                    if dep != []:
                        deps.append(dep)
                        dep = []
                if flag:
                    dep.append(line.split('\t'))

            wordnet = WikiWordnet()

            positive = {}
            negative = {}

            for dep in deps:
                dic = {}
                for line in dep:
                    score = self._Score(line[2], wordnet, self._workbook)
                    reverse = []
                    if score != 0:
                        if (line[3] == 'NOUN'or 'ADJ' or 'ADV') and len(line) > 5:
                             if dep[int(line[6])-1][3] != 'ADP' and dep[int(line[6])-1][3] != 'PUNCT':
                                self._Add(dep[int(line[6])-1][2], dic, score)
                        elif line[3] == 'VERB': 
                            for l in dep:
                                if len(l) > 5 and dep[int(l[6])-1] == line and (l[7] == 'nsubj' or 'obj'):
                                    self._Add(l[2], dic, score)

                for l in dep:
                    if l[2] == u'не' and len(l) > 5 and dep[int(l[6])-1][2] in dic:
                        dic[dep[int(l[6])-1][2]] *= -1
                        
                self._Distr(dictionary = dic, neg_dic = negative, pos_dic = positive)

            self._negatives.append(negative)
            self._positives.append(positive)

In [5]:
class SOExtracter(Parser):
    _model = None
    _objects = []
    _subjects = []
    
    def _Add(dictionary, word):
        if word in dictionary:
            dictionary[word] += 1
        else:
            dictionary[word] = 1

    def __init__(self, data_path, model_path):
        Parser.__init__(self, data_path)
        self._model = Model(model_path)
    def get_objects(self):
        return self._objects
    def get_subjects(self):
        return self._subjects
    def extract_so(self):
        for i, doc in enumerate(self._data):
            sentences = model.tokenize(doc)
            for s in sentences:
                self._model.tag(s)
                self._model.parse(s)
            conllu = self._model.write(sentences, "conllu")

            flag = False
            deps = []
            dep = []
            for line in conllu.split('\n'):
                if len(line) > 1 and line[0] == '1':
                    flag = True
                elif len(line) == 0:
                    flag = False
                    if dep != []:
                        deps.append(dep)
                        dep = []
                if flag:
                    dep.append(line.split('\t'))

            subjects = {}
            objects = {}
            for dep in deps:
                adjects = {}
                verbs = {}
                appos = {}
                for line in dep:
                    if line[3] == 'VERB':
                        verbs[line[0]] = ['0', line[2], '0']
                    elif line[3] == 'ADJ':
                        if len(dep[int(line[6])-1]) > 5 and dep[int(line[6])-1][3] == 'NOUN':
                            adjects[line[0]] = [dep[int(line[6])-1][2], '0', line[2]]
                for line in dep:
                    if len(line) > 6 and line[7] == 'nsubj':
                        if line[6] in verbs:
                            verbs[line[6]][0] = line[2]
                    elif len(line) > 6 and (line[7] == 'obj' or line[7] == 'iobj'):
                        if line[6] in verbs:
                            verbs[line[6]][2] = line[2]
                    elif len(line) > 6 and line[7] == 'appos':
                        appos[line[0]] = [line[2], '0', dep[int(line[6])-1][2]]
                for key in verbs:
                    if verbs[key][0] != '0':
                        self._Add(objects, verbs[key][0].lower())
                    if verbs[key][2] != '0':
                        self._Add(subjects, verbs[key][2].lower())
                for key in appos:
                    if appos[key][0] != '0':
                        self._Add(objects, appos[key][0].lower())
                    if appos[key][2] != '0':
                        self._Add(subjects, appos[key][2].lower())
                for key in adjects:
                    if adjects[key][0] != '0':
                        self._Add(objects, adjects[key][0].lower())
                    if adjects[key][2] != '0':
                        self._Add(subjects, adjects[key][2].lower())

            self._subjects.append(subjects)
            self._objects.append(objects)

In [6]:
class NGExtracter(Parser):
    _ngramms = []
    _morph = pymorphy2.MorphAnalyzer()
    _ng = ngrammer.NGrammer()
    _delimiters_regex = [u'[^a-zа-яё ]+']
    _stopwords = set(stopwords.words('russian') + [u'тот', u'те', u'та', u'то', u'этот', u'эти', u'эта', u'это',u'он', u'она', u'оно', u'наш', u'ваш', u'быть', u'мы', u'ты', u'вы', u'там', u'свой', u'своя', u'своё', u'ещё'])
    def __init__(self, data_path, stopwords=_stopwords, delimiters_regex=_delimiters_regex):
        Parser.__init__(self, data_path)
        _stopwords = stopwords
        _delimiters_regex = delimiters_regex
    def get_ngramms(self):
        return self._ngramms
    def fit_ngrammer(self, treshhold, n):
        corpus_text = []
        self._ng.delimiters = list(self._stopwords)
        delimiters_regex = [u'[^a-zа-яё ]+']
        self._ng.delimiters_regex = self._delimiters_regex

        for doc in self._data:
            tokens = nltk.word_tokenize(doc)
            words = []
            for token in tokens:
                word = self._morph.parse(token)[0].normal_form
                if word.isalpha() and word not in self._stopwords:
                    words.append(word)
            corpus_text.append(words)
        self._ng.frequentPhraseMining(corpus_text, threshhold=treshhold, max_ngramm_len=n)
    
    def extract_ngramms(self, treshhold):
        for doc in self._data:
            tokens = nltk.word_tokenize(doc)
            words = []
            for token in tokens:
                word = self._morph.parse(token)[0].normal_form
                if word.isalpha() and word not in self._stopwords:
                    words.append(word)   

            res = self._ng.ngramm(words, threshhold=treshhold)
            self._ngramms.append(res[0])
            

In [7]:
ngex = NGExtracter('data_DNR')


In [8]:
ngex.fit_ngrammer(15, 2)

In [9]:
ngex.extract_ngramms(1)

In [10]:
for d in ngex.get_ngramms():
    for ng in d:
        print ng

национализация
украинский_предприятие
который
объявить
донецкий
народный_республика
луганский
народный_республика
сегодня
ответ
действие
киев
цель
выжить
заявить
президент_рф
дмитрий_песок
являться
свидетель
отторгнуть_государство
область
попадать
тяжёлый_положение
условие_полный
блокада_сторона
экстремистский
элемент
поэтому
степень_понять
действие
руководство_регион
который
отторгнуть_государство
речь_идти
жизнь_несколько
миллион_человек
человек
выживать
сказать_песок
стать
отвечать
вопрос
открытый
российский_рынок
национализировать
донбасс
предприятие
также_отметить
москва
пытаться
использовать
влияние
лнр_днр
нормализация
обстановка
регион
однако
безгранично
март
наступить
украинский
радикал
блокада
днр_лнр
снять
весь
идти_план
предприятие
сегодняшний_день
нуль
часы
вводиться
внешний_управление
сообщить
утром
лидер_днр
александр_захарченко
принцип
сегодняшний
вечер
весь
закончить
корреспондент
отправиться
самый
известный
донецкий
шахта_имя
засядько
посмотреть
выглядеть
процесс
смен

стрелковый
оружие
боевик
обстрелять
авдеевка
шахта
бутовка
троицкий
песок
верхнеторецкий
невельский
опытный
зайцевый
боевик
обстрелять
кп
марьинка
аргумент
нападение
крым
украсть
миллион
погибнуть
украсть
человек
кремль_считать
некорректный
сравнивать
национализация_предприятие
самопровозгласить
днр_лнр
отжим
рассказать
журналист
президент
дмитрий_песок
сообщать
риа
новость_думать
слово_отжим
принцип
являться
свидетель
отторгнуть_государство
область
попадать
тяжёлый_положение
условие_полный
блокада_сторона
экстремистский
элемент
поэтому
степень_понять
действие_который
предприниматься
руководство_регион
который
повторять
отторгнуть_государство
речь_идти
жизнь_несколько
миллион_человек
поэтому
безусловно
человек
выживать
сказать_песок
укртелек
связь
национализация
украинский_предприятие
российский_боевик
дистанционно
отключить
оборудование
оккупировать_территория
крупный_украинский
оператор
фиксировать
телефонный_связь
укртелек
март_год
прекратить
обслуживание
абонент
территория_контроли

KeyboardInterrupt: 